In [1]:
%load_ext autoreload

In [2]:
import wandb
import random

In [3]:
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="my-awesome-project",
#     entity="llmnerds",
#     # track hyperparameters and run metadata
#     config={
#         "learning_rate": 0.02,
#         "architecture": "CNN",
#         "dataset": "CIFAR-100",
#         "epochs": 10,
#         "sdfsfs": 12,
#     },
# )

# # simulate training
# epochs = 10
# offset = random.random() / 5


# for epoch in range(2, epochs):
#     acc = 1 - 2**-epoch - random.random() / epoch - offset
#     loss = 2**-epoch + random.random() / epoch + offset

#     # log metrics to wandb
#     wandb.log({"acc": acc, "loss": loss}, step=epoch)
#     wandb.log({"val_acc": acc, "val_loss": loss}, step=epoch)

# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xl0 (llmnerds). Use `wandb login --relogin` to force relogin


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


acc,▁▃▇█▇▇██
loss,▇█▁▃▂▁▂▁
val_acc,▁▃▇█▇▇██
val_loss,▇█▁▃▂▁▂▁
acc,0.82512
loss,0.20847
val_acc,0.82512
val_loss,0.20847


In [4]:
%autoreload 2

from lovely_tensors.patch import monkey_patch; monkey_patch()
import torch
from transformers import GPT2Tokenizer

In [5]:
import math


def new_gelu(input):
    return (
        0.5
        * input
        * (
            1.0
            + torch.tanh(
                math.sqrt(2.0 / math.pi) * (input + 0.044715 * torch.pow(input, 3.0))
            )
        )
    )


def conv_1d(x, weight, bias=None):
    size_out = x.size()[:-1] + (weight.size(-1),)
    x = torch.addmm(bias, x.view(-1, x.size(-1)), weight)
    x = x.view(size_out)
    return x


def transformer_block(i, input_hidden_state, model_state):
    def block_state(key):
        return model_state[f"h.{i}.{key}"]

    def attention_state(key):
        return model_state[f"h.{i}.attn.{key}"]

    # attention block
    ln1 = torch.nn.functional.layer_norm(
        input=input_hidden_state,
        weight=block_state("ln_1.weight"),
        bias=block_state("ln_1.bias"),
        normalized_shape=(768,),
    )

    w_q, w_k, w_v = attention_state("c_attn.weight").chunk(3, dim=1)
    b_q, b_k, b_v = attention_state("c_attn.bias").chunk(3, dim=0)

    q = conv_1d(ln1, w_q, b_q)
    k = conv_1d(ln1, w_k, b_k)
    v = conv_1d(ln1, w_v, b_v)

    q_chunked = torch.stack(q.chunk(12, dim=-1))
    k_chunked = torch.stack(k.chunk(12, dim=-1))
    v_chunked = torch.stack(v.chunk(12, dim=-1))

    attention = torch.matmul(q_chunked, k_chunked.transpose(-1, -2))

    attention_rescaled = attention / (64**0.5)

    mask = torch.triu(torch.ones_like(attention_rescaled), diagonal=1).bool()
    attention_masked = attention_rescaled.masked_fill(
        mask, torch.finfo(torch.float32).min
    )

    attention_softmaxed = torch.nn.functional.softmax(attention_masked, dim=-1)
    attention_output = torch.matmul(attention_softmaxed, v_chunked)

    out_tuple = [x[0] for x in attention_output.chunk(12, dim=0)]
    combined_attention_output = torch.cat(out_tuple, dim=-1)

    w_cproj = attention_state("c_proj.weight")
    b_cproj = attention_state("c_proj.bias")

    crosstalk = conv_1d(combined_attention_output, w_cproj, b_cproj)
    after_residual = crosstalk + input_hidden_state

    # mlp block
    before_ln2 = after_residual

    ln2 = torch.nn.functional.layer_norm(
        input=after_residual,
        weight=block_state("ln_2.weight"),
        bias=block_state("ln_2.bias"),
        normalized_shape=(768,),
    )

    w_fc = block_state("mlp.c_fc.weight")
    b_fc = block_state("mlp.c_fc.bias")

    after_up = conv_1d(ln2, w_fc, b_fc)
    activated = new_gelu(after_up)

    w_proj = block_state("mlp.c_proj.weight")
    b_proj = block_state("mlp.c_proj.bias")

    after_down = conv_1d(activated, w_proj, b_proj)

    after_residual_2 = after_down + before_ln2

    return after_residual_2


def transformer(token_ids, model_state):
    token_embeddings = model_state["wte.weight"][token_ids]
    positions = torch.arange(len(token_ids))  # [0,1,2,3...]
    position_embeddings = model_state["wpe.weight"][positions]
    embeddings = token_embeddings + position_embeddings

    hs = embeddings
    for i in range(12):
        hs = transformer_block(i, hs, model_state)

    ln_w = model_state["ln_f.weight"]
    ln_b = model_state["ln_f.bias"]

    ln = torch.nn.functional.layer_norm(
        input=hs, weight=ln_w, bias=ln_b, normalized_shape=(768,)
    )

    return ln

In [6]:
def language_model(token_ids, model_state):
    transformer_output = transformer(token_ids, model_state)
    logits = torch.matmul(transformer_output, model_state["wte.weight"].T)
    return logits

In [7]:
token_ids = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
model_params = torch.load("pytorch_model.bin")

In [8]:
out = language_model(token_ids, model_params)
out

# out[0].chans(scale=4, cmap="seismic")
# out[1].chans(scale=4, cmap="seismic")
# out[2].chans(scale=4, cmap="seismic")
# out[3]

tensor[10, 50257] n=502570 (1.9Mb) x∈[-110.039, -29.114] μ=-76.757 σ=15.804

In [9]:
with open("tiny_shakespeare.txt", "r") as f:
    data = f.read()
chars = sorted(list(set(data)))

# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}


def encode(s):
    return [stoi[c] for c in s]  # encoder: take a string, output a list of integers


def decode(l):
    return "".join(
        [itos[i] for i in l]
    )  # decoder: take a list of integers, output a string


encoded_data = encode(data)

In [10]:
train_data = encoded_data[: int(len(encoded_data) * 0.8)]
val_data = encoded_data[int(len(encoded_data) * 0.8) :]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data = torch.tensor(train_data).to(device)
val_data = torch.tensor(val_data).to(device)

In [11]:
892315 / (5 * 32)

5576.96875

In [12]:
ctx = 5


def get_item(data, ctx):
    # i = random.randint(0, len(data) - ctx - 1)
    i = 0
    while i + ctx < len(data):
        src = data[i : i + ctx]
        dst = data[i + 1 : i + ctx + 1]
        yield src, dst
        i += ctx


def get_batch(data, ctx_len, batch_size):
    """Yields a tuple of tensors of shape (batch_size, ctx).
    X, shape=B C
    y, shape=B C
    """
    gen = get_item(data, ctx_len)

    try:
        while True:
            X, y = zip(*[next(gen) for _ in range(batch_size)])
            yield torch.stack(X), torch.stack(y)
    except StopIteration:
        pass


for batch in get_batch(train_data[:100], ctx_len=5, batch_size=2):
    print(batch)

(tensor[2, 5] i64 n=10 x∈[1, 58] μ=40.400 σ=20.998 cuda:0 [[18, 47, 56, 57, 58], [1, 15, 47, 58, 47]], tensor[2, 5] i64 n=10 x∈[1, 64] μ=45.000 σ=20.580 cuda:0 [[47, 56, 57, 58, 1], [15, 47, 58, 47, 64]])
(tensor[2, 5] i64 n=10 x∈[0, 64] μ=37.900 σ=21.886 cuda:0 [[64, 43, 52, 10, 0], [14, 43, 44, 53, 56]], tensor[2, 5] i64 n=10 x∈[0, 56] μ=35.800 σ=20.032 cuda:0 [[43, 52, 10, 0, 14], [43, 44, 53, 56, 43]])
(tensor[2, 5] i64 n=10 x∈[1, 61] μ=39.600 σ=21.423 cuda:0 [[43, 1, 61, 43, 1], [54, 56, 53, 41, 43]], tensor[2, 5] i64 n=10 x∈[1, 61] μ=39.600 σ=21.423 cuda:0 [[1, 61, 43, 1, 54], [56, 53, 41, 43, 43]])
(tensor[2, 5] i64 n=10 x∈[1, 63] μ=40.000 σ=22.015 cuda:0 [[43, 42, 1, 39, 52], [63, 1, 44, 59, 56]], tensor[2, 5] i64 n=10 x∈[1, 63] μ=41.500 σ=22.741 cuda:0 [[42, 1, 39, 52, 63], [1, 44, 59, 56, 58]])
(tensor[2, 5] i64 n=10 x∈[1, 58] μ=39.400 σ=20.001 cuda:0 [[58, 46, 43, 56, 6], [1, 46, 43, 39, 56]], tensor[2, 5] i64 n=10 x∈[1, 56] μ=33.700 σ=22.121 cuda:0 [[46, 43, 56, 6, 1], [46,

In [17]:
torch.random.manual_seed(42)

CTX_LEN = 1


# in_ # BATCH_SIZE x CTX_LEN x len(chars)  (feed in a batch of CTX_LEN embeddings (each is a one-hot-encoded character)
# out_ # BATCH_SIZE x len(chars)  (get out a (one-hot-encoded) next-char prediction for each batch-item
l1 = torch.nn.Linear(len(chars) * CTX_LEN, len(chars)).to(device)

model_params = {
    "w": l1.weight,
    "b": l1.bias,
}


def model(params, input_ids, vocab_size):
    """This model takes in a sequence and predicts 1 token"""

    one_hot_inputs = torch.nn.functional.one_hot(input_ids, num_classes=vocab_size)
    one_hot_inputs = one_hot_inputs.float()

    combined_ont_hot_inputs = one_hot_inputs.reshape(
        (input_ids.shape[0], -1)
    )  # BATCH_SIZE x (CTX_LEN * len(chars))

    preds = combined_ont_hot_inputs @ params["w"].transpose(-2, -1) + params["b"]

    # preds = torch.rand(
    #     (input_ids.shape[0], vocab_size), dtype=torch.float32, device=input_ids.device
    # )

    return preds

batch_gen = get_batch(train_data, ctx_len=CTX_LEN, batch_size=32)
X, y = next(batch_gen)
with torch.no_grad():
    preds = model(model_params, X, vocab_size=len(chars))
    loss = torch.nn.functional.cross_entropy(preds, y[:, -1])
loss

tensor cuda:0 4.218

In [19]:
from tqdm.auto import tqdm

BATCH_SIZE = 32
NEPOCH = 10

LOG_INTERVAL = 500
VALIDATION_INTERVAL = 1_250

wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",
    entity="llmnerds",
    config={
        "batch_size": BATCH_SIZE,
        "ctx": CTX_LEN,
    },
)

l1 = torch.nn.Linear(len(chars) * CTX_LEN, len(chars)).to(device)
model_params = {
    "w": l1.weight,
    "b": l1.bias,
}

i = 1
total_loss = 0
val_total_loss = 0
for epoch in range(NEPOCH):
    for X, y in tqdm(get_batch(train_data, ctx_len=CTX_LEN, batch_size=BATCH_SIZE)):
        preds = model(params=model_params, input_ids=X, vocab_size=len(chars))
        loss = torch.nn.functional.cross_entropy(preds, y[:, -1])
        total_loss += loss.item()
        loss.backward()

        with torch.no_grad():
            for param in model_params.values():
                param -= 1 * param.grad
                param.grad.zero_()

        if i % LOG_INTERVAL == 0:
            wandb.log({"loss": total_loss / LOG_INTERVAL}, step=i)
            total_loss = 0

        if i % VALIDATION_INTERVAL == 0:
            j = 0
            for X, y in get_batch(val_data, ctx_len=CTX_LEN, batch_size=BATCH_SIZE):
                preds = model(params=model_params, input_ids=X, vocab_size=len(chars))
                loss = torch.nn.functional.cross_entropy(preds, y[:, -1])
                val_total_loss += loss.item()
                j += 1
            wandb.log({"val_loss": val_total_loss / j}, step=i)
            val_total_loss = 0
        i += 1

wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


loss,█▁
loss,2.70234


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
X  # abc

In [ ]:
y  # d

In [ ]:
preds

In [ ]:
# from torch import tensor

# y = tensor([range(65)])  # y
# y_hat = tensor.rand  # preds

# ce_loss = -torch.log(y_hat[y == 1]).mean()
# ce_loss

import torch
import torch.nn as nn

y = torch.tensor([3])  # Class index
# y_hat = torch.tensor([[0.1, 0.1, 0.1, 0.5, 0.1, 0.1]])  # Logits
y_hat = torch.rand((1, 65)).float()  # Logits

loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(y_hat, y)
loss

In [ ]:
import numpy as np

A = np.array([1, 2, 3, 4, 5])
A == 2

In [ ]:
B = np.array([10, 20, 30, 40, 50])
B[A == 2]